# Comparative Analysis of Sequence-to-Sequence Models for Urdu Poetry Generation

**Deep Learning for Perception - Semester Project Part 2**

**Project Requirements:**
- Dataset: Urdu Poetry Dataset from Hugging Face
- Models: SimpleRNN, LSTM, Transformer
- Optimizers: Adam, RMSprop, SGD
- Total Experiments: 9 combinations (3 models × 3 optimizers)
- Hyperparameter Tuning: OFAT methodology
- Output: Structured results for LaTeX report

**Notebook Structure:**
1. STEP 1: Dataset Loading & Exploration
2. STEP 2: Data Preprocessing & Tokenization
3. STEP 3: Model Architecture Definitions
4. STEP 4: Optimizer Comparison (9 experiments)
5. STEP 5: Hyperparameter Tuning
6. STEP 6: Text Generation & Evaluation
7. STEP 7: Results Export & Visualization

## 📋 Implementation Notes

**Based on Experimental Results Analysis:**
- Initial models showed poor performance (4-9% accuracy, 700-1100 perplexity)
- Root cause: Vocabulary too large for dataset size (1300 poems)
- Solution: Reduced complexity, optimized for limited data

**Applied Optimizations:**
- Vocabulary: 1000 tokens (min_freq=5)
- Embedding: 128 dimensions
- RNN/LSTM units: 256
- Sequence length: 20 tokens
- Dropout: 0.1 (reduced from 0.2)
- Training: 10 epochs, batch_size=64
- Early stopping: patience=3

**Expected Performance:**
- Accuracy: 20-40% (improved from 4-9%)
- Perplexity: 100-300 (improved from 700-1100)
- Training Speed: 2-3x faster

---

## STEP 0: Environment Setup

Install required packages.

In [ ]:
# Install required packages
# !pip install pandas matplotlib scikit-learn tensorflow

## STEP 1: Dataset Loading & Exploration

Load the Urdu poetry dataset and perform basic exploration.

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Load dataset
df = pd.read_csv("/kaggle/input/poetry/output_ur.csv")

# Basic exploration
print("=== Dataset Overview ===")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Basic dataset info
poem_col = 'content'
print("\n=== Dataset Structure ===")
print(f"Total poems: {len(df)}")
print(f"Missing values: {df[poem_col].isnull().sum()}")
print(f"Duplicate poems: {df.duplicated(subset=[poem_col]).sum()}")
print("\nData loading complete!")

=== Dataset Overview ===
Shape: (1323, 2)
Columns: ['title', 'content']

=== Dataset Structure ===
Total poems: 1323
Missing values: 9
Duplicate poems: 8

Data loading complete!


## STEP 2: Data Preprocessing & Tokenization

Clean, normalize, tokenize, and create sequences with strict 80-10-10 split.

In [31]:
import re
import unicodedata
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

def clean_text(text):
    """
    Clean and normalize Urdu text
    """
    if pd.isna(text):
        return ""

    # Convert to string
    text = str(text)

    # Normalize Unicode (handle different representations of same characters)
    text = unicodedata.normalize('NFC', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text.strip())

    # Remove control characters but keep Urdu characters
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)

    # Handle common Urdu special characters - keep Urdu script and basic punctuation
    # Remove non-Urdu characters except spaces and basic punctuation
    text = re.sub(r'[^\u0600-\u06FF\s\.,!?؛:۔]', '', text)

    return text.strip()

def normalize_urdu_text(text):
    """
    Additional Urdu-specific normalization
    """
    if not text:
        return ""

    # Handle Urdu specific character variations
    # Normalize alef variants
    text = re.sub(r'[اأإآ]', 'ا', text)

    # Normalize yeh variants
    text = re.sub(r'[يى]', 'ی', text)

    # Normalize heh variants
    text = re.sub(r'[هة]', 'ہ', text)

    return text

def tokenize_text(text):
    """
    Tokenize Urdu text
    """
    if not text:
        return []

    # Simple tokenization by whitespace
    tokens = text.split()

    return tokens

def preprocess_data(df, poem_col='content', remove_duplicates=True, remove_missing=True):
    """
    Main preprocessing pipeline with improved data quality handling
    """
    print("Starting data preprocessing...")
    
    initial_count = len(df)
    
    # Remove missing values first
    if remove_missing:
        print("Removing missing values...")
        df = df.dropna(subset=[poem_col]).copy()
        print(f"Removed {initial_count - len(df)} rows with missing values")
    
    # Remove duplicates
    if remove_duplicates:
        print("Removing duplicate poems...")
        before_dedup = len(df)
        df = df.drop_duplicates(subset=[poem_col]).copy()
        print(f"Removed {before_dedup - len(df)} duplicate poems")

    # Clean text
    print("Cleaning text...")
    df['cleaned_text'] = df[poem_col].apply(clean_text)

    # Normalize Urdu text
    print("Normalizing Urdu text...")
    df['normalized_text'] = df['cleaned_text'].apply(normalize_urdu_text)

    # Filter empty lines
    print("Filtering empty texts...")
    df = df[df['normalized_text'].str.len() > 0].copy()

    # Tokenize
    print("Tokenizing text...")
    df['tokens'] = df['normalized_text'].apply(tokenize_text)

    # Filter out empty token lists and very short sequences
    df = df[df['tokens'].str.len() > 5].copy()  # At least 5 tokens

    print(f"Preprocessing complete. Remaining samples: {len(df)}")

    return df

def build_vocabulary(tokens_list, min_freq=2, max_vocab_size=5000):
    """
    Build vocabulary from tokenized texts with frequency filtering
    """
    print("Building vocabulary...")

    # Flatten all tokens
    all_tokens = [token for tokens in tokens_list for token in tokens]

    # Count frequencies
    token_counts = Counter(all_tokens)

    # Filter by minimum frequency
    vocab = {token: count for token, count in token_counts.items() if count >= min_freq}

    # Sort by frequency (most common first)
    vocab = dict(sorted(vocab.items(), key=lambda x: x[1], reverse=True))
    
    # Limit vocabulary size (keep most frequent tokens)
    if len(vocab) > max_vocab_size:
        vocab = dict(list(vocab.items())[:max_vocab_size])
        print(f"Vocabulary limited to top {max_vocab_size} most frequent tokens")

    # Add special tokens
    special_tokens = {
        '<PAD>': 0,  # Padding token (use index 0 for padding)
        '<UNK>': 1,  # Unknown token
        '<SOS>': 2,  # Start of sequence
        '<EOS>': 3   # End of sequence
    }

    # Create token to index mapping (start from 4 to reserve 0-3 for special tokens)
    token_to_idx = special_tokens.copy()
    for idx, token in enumerate(vocab.keys(), start=4):
        token_to_idx[token] = idx

    # Create index to token mapping
    idx_to_token = {idx: token for token, idx in token_to_idx.items()}

    print(f"Vocabulary size: {len(token_to_idx)} (including special tokens)")
    print(f"Special tokens: {list(special_tokens.keys())}")

    return token_to_idx, idx_to_token, vocab

def prepare_sequences(tokens_list, token_to_idx, max_length=None):
    """
    Convert tokens to numerical sequences
    """
    print("Preparing sequences...")

    sequences = []
    for tokens in tokens_list:
        # Convert tokens to indices
        seq = [token_to_idx.get(token, token_to_idx['<UNK>']) for token in tokens]

        # Add start and end tokens
        seq = [token_to_idx['<SOS>']] + seq + [token_to_idx['<EOS>']]

        sequences.append(seq)

    return sequences

def split_data(df, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, random_state=42):
    """
    Split data into train, validation, and test sets
    """
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    print("Splitting data...")

    # First split: train and temp (val+test)
    train_df, temp_df = train_test_split(
        df,
        train_size=train_ratio,
        random_state=random_state,
        shuffle=True
    )

    # Second split: val and test from temp
    val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
    val_df, test_df = train_test_split(
        temp_df,
        train_size=val_ratio_adjusted,
        random_state=random_state,
        shuffle=True
    )

    print(f"Train set: {len(train_df)} samples")
    print(f"Validation set: {len(val_df)} samples")
    print(f"Test set: {len(test_df)} samples")

    return train_df, val_df, test_df

# Execute preprocessing with improved settings
print("=" * 60)
print("IMPROVED PREPROCESSING PIPELINE")
print("=" * 60)

processed_df = preprocess_data(df, poem_col, remove_duplicates=True, remove_missing=True)

# Build vocabulary - reduced size for better learning with limited data
token_to_idx, idx_to_token, vocab = build_vocabulary(
    processed_df['tokens'],
    min_freq=5,  # Higher frequency threshold to reduce noise
    max_vocab_size=1000  # Smaller vocab for 1300 poems
)

# Prepare sequences
sequences = prepare_sequences(processed_df['tokens'], token_to_idx)
processed_df['sequences'] = sequences

# Split data
train_df, val_df, test_df = split_data(processed_df)
train_sequences = train_df['sequences'].tolist()
val_sequences = val_df['sequences'].tolist()
test_sequences = test_df['sequences'].tolist()

# Save processed data
processed_data = {
    'train_sequences': train_sequences,
    'val_sequences': val_sequences,
    'test_sequences': test_sequences,
    'token_to_idx': token_to_idx,
    'idx_to_token': idx_to_token,
    'vocab': vocab
}

with open('processed_data.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

print("Preprocessing complete!")
print(f"Vocab: {len(token_to_idx)} | Train: {len(train_sequences)} | Val: {len(val_sequences)} | Test: {len(test_sequences)}")

IMPROVED PREPROCESSING PIPELINE
Starting data preprocessing...
Removing missing values...
Removed 9 rows with missing values
Removing duplicate poems...
Removed 0 duplicate poems
Cleaning text...
Normalizing Urdu text...
Filtering empty texts...
Tokenizing text...
Preprocessing complete. Remaining samples: 1314
Building vocabulary...
Vocabulary limited to top 1000 most frequent tokens
Vocabulary size: 1004 (including special tokens)
Special tokens: ['<PAD>', '<UNK>', '<SOS>', '<EOS>']
Preparing sequences...
Splitting data...
Train set: 1051 samples
Validation set: 131 samples
Test set: 132 samples
Preprocessing complete!
Vocab: 1004 | Train: 1051 | Val: 131 | Test: 132


## STEP 3: Model Architecture Definitions

Define SimpleRNN, LSTM, and Transformer architectures.

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout, Input, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import numpy as np

def create_rnn_model(vocab_size, embedding_dim, rnn_units, seq_length):
    """
    Create improved RNN-based model for text generation
    """
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=seq_length, mask_zero=True),
        SimpleRNN(rnn_units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1),
        SimpleRNN(rnn_units, dropout=0.1, recurrent_dropout=0.1),
        Dense(rnn_units // 2, activation='relu'),
        Dropout(0.2),
        Dense(vocab_size, activation='softmax')
    ])

    return model

def create_lstm_model(vocab_size, embedding_dim, rnn_units, seq_length):
    """
    Create improved LSTM-based model for text generation
    """
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=seq_length, mask_zero=True),
        LSTM(rnn_units, return_sequences=True, dropout=0.1, recurrent_dropout=0.0),
        LSTM(rnn_units, dropout=0.1, recurrent_dropout=0.0),
        Dense(rnn_units // 2, activation='relu'),
        Dropout(0.2),
        Dense(vocab_size, activation='softmax')
    ])

    return model

def create_transformer_block(x, embed_dim, num_heads, ff_dim, dropout_rate=0.05):
    """
    Transformer encoder block
    """
    # Multi-head attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim // num_heads)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed-forward network (must output same dimension as input)
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dense(embed_dim)(ffn_output)  # Project back to embed_dim
    ffn_output = Dropout(dropout_rate)(ffn_output)
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

    return out2

def create_transformer_model(vocab_size, embedding_dim, num_heads, ff_dim, num_blocks, seq_length, dropout_rate=0.05):
    """
    Create improved Transformer-based model for text generation
    """
    inputs = Input(shape=(seq_length,))

    # Embedding layer
    x = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs)

    # Positional encoding (simplified)
    positions = tf.range(start=0, limit=seq_length, delta=1)
    positional_encoding = Embedding(seq_length, embedding_dim)(positions)
    x = x + positional_encoding

    # Transformer blocks
    for _ in range(num_blocks):
        x = create_transformer_block(x, embedding_dim, num_heads, ff_dim, dropout_rate)

    # Global pooling and output
    x = GlobalAveragePooling1D()(x)
    x = Dense(ff_dim, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(vocab_size, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

def get_optimizer(optimizer_name, learning_rate=0.001):
    """
    Get optimizer by name with appropriate settings
    """
    optimizers = {
        'adam': Adam(learning_rate=learning_rate, clipnorm=1.0),
        'rmsprop': RMSprop(learning_rate=learning_rate, clipnorm=1.0),
        'sgd': SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True, clipnorm=1.0)
    }

    return optimizers.get(optimizer_name.lower(), Adam(learning_rate=learning_rate))

def create_model(model_type, vocab_size, embedding_dim, rnn_units, seq_length,
                num_heads=8, ff_dim=128, num_blocks=2):
    """
    Factory function to create models
    """
    if model_type.lower() == 'rnn':
        return create_rnn_model(vocab_size, embedding_dim, rnn_units, seq_length)
    elif model_type.lower() == 'lstm':
        return create_lstm_model(vocab_size, embedding_dim, rnn_units, seq_length)
    elif model_type.lower() == 'transformer':
        return create_transformer_model(vocab_size, embedding_dim, num_heads, ff_dim, num_blocks, seq_length)
    else:
        raise ValueError(f"Unknown model type: {model_type}")

def compile_model(model, optimizer_name, learning_rate=0.001):
    """
    Compile model with specified optimizer
    """
    optimizer = get_optimizer(optimizer_name, learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Optimized model configurations for small dataset (1300 poems)
MODEL_CONFIGS = {
    'rnn': {
        'embedding_dim': 128,  # Reduced to prevent overfitting
        'rnn_units': 256,      # Reduced capacity
        'seq_length': 20,      # Shorter sequences for faster training
        'learning_rate': 0.001
    },
    'lstm': {
        'embedding_dim': 128,
        'rnn_units': 256,
        'seq_length': 20,
        'learning_rate': 0.001
    },
    'transformer': {
        'embedding_dim': 128,
        'num_heads': 4,        # Reduced from 8
        'ff_dim': 256,         # Reduced from 512
        'num_blocks': 2,       # Reduced from 3
        'seq_length': 20,
        'learning_rate': 0.001  # Increased for faster convergence
    }
}

OPTIMIZERS = ['adam', 'rmsprop', 'sgd']

print("Improved model definitions loaded!")
print(f"Available models: {list(MODEL_CONFIGS.keys())}")
print(f"Available optimizers: {OPTIMIZERS}")
print("\nOptimizations for small dataset (1300 poems):")
print("  • Reduced embedding: 256 → 128 dimensions")
print("  • Reduced RNN/LSTM units: 512 → 256")
print("  • Reduced sequence length: 40 → 20 tokens")
print("  • Lower dropout: 0.2 → 0.1 (was blocking learning)")
print("  • Transformer: 4 heads, 256 ff_dim, 2 blocks")
print("  • Faster training: 20 epochs, batch_size=64")

Improved model definitions loaded!
Available models: ['rnn', 'lstm', 'transformer']
Available optimizers: ['adam', 'rmsprop', 'sgd']

Optimizations for small dataset (1300 poems):
  • Reduced embedding: 256 → 128 dimensions
  • Reduced RNN/LSTM units: 512 → 256
  • Reduced sequence length: 40 → 20 tokens
  • Lower dropout: 0.2 → 0.1 (was blocking learning)
  • Transformer: 4 heads, 256 ff_dim, 2 blocks
  • Faster training: 10 epochs, batch_size=64


## STEP 4: Optimizer Comparison (Core Experiments)

Train and evaluate 9 model-optimizer combinations. Each experiment runs in a separate cell for progress tracking and memory management.

**Required Combinations:**
1. SimpleRNN + Adam
2. SimpleRNN + RMSprop  
3. SimpleRNN + SGD
4. LSTM + Adam
5. LSTM + RMSprop
6. LSTM + SGD
7. Transformer + Adam
8. Transformer + RMSprop
9. Transformer + SGD

**Metrics Tracked:** Training time, accuracy, loss, perplexity (exp(loss))

In [41]:
import time
from datetime import datetime
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Global results storage for all experiments
if 'all_experiment_results' not in globals():
    all_experiment_results = []

def prepare_training_data(sequences, seq_length, vocab_size):
    """Prepare input-target pairs with padding"""
    X, y = [], []
    
    for seq in sequences:
        if len(seq) > seq_length + 1:
            for i in range(len(seq) - seq_length):
                X.append(seq[i:i+seq_length])
                y.append(seq[i+seq_length])
        elif len(seq) > 1:
            padded_seq = seq[:-1]
            if len(padded_seq) < seq_length:
                padded_seq = padded_seq + [0] * (seq_length - len(padded_seq))
            elif len(padded_seq) > seq_length:
                padded_seq = padded_seq[:seq_length]
            X.append(padded_seq)
            y.append(seq[-1])
    
    X_padded = pad_sequences(X, maxlen=seq_length, padding='pre', value=0)
    y_array = np.array(y)
    
    print(f"Training samples: {len(X_padded)}, Input shape: {X_padded.shape}")
    return X_padded, y_array

def calculate_perplexity(loss):
    """Calculate perplexity from cross-entropy loss"""
    import math
    try:
        return math.exp(loss)
    except:
        return float('inf')

def train_and_evaluate(model_type, optimizer_name, train_sequences, val_sequences, test_sequences,
                       token_to_idx, epochs=20, batch_size=64, learning_rate=None, experiment_name=None):
    """
    Train and evaluate a model-optimizer combination.
    Tracks all metrics for CSV export.
    """
    print("=" * 80)
    print(f"EXPERIMENT: {model_type.upper()} + {optimizer_name.upper()}")
    if experiment_name:
        print(f"Custom Config: {experiment_name}")
    print("=" * 80)
    
    config = MODEL_CONFIGS[model_type.lower()]
    vocab_size = len(token_to_idx)
    seq_length = config['seq_length']
    
    if learning_rate is None:
        learning_rate = config.get('learning_rate', 0.001)
    
    # Prepare data
    print("\nPreparing data...")
    X_train, y_train = prepare_training_data(train_sequences, seq_length, vocab_size)
    X_val, y_val = prepare_training_data(val_sequences, seq_length, vocab_size)
    X_test, y_test = prepare_training_data(test_sequences, seq_length, vocab_size)
    
    # Create model
    print(f"\nBuilding {model_type.upper()} model...")
    if model_type.lower() == 'transformer':
        model = create_transformer_model(
            vocab_size=vocab_size,
            embedding_dim=config['embedding_dim'],
            num_heads=config['num_heads'],
            ff_dim=config['ff_dim'],
            num_blocks=config['num_blocks'],
            seq_length=seq_length
        )
    else:
        model = create_model(
            model_type=model_type,
            vocab_size=vocab_size,
            embedding_dim=config['embedding_dim'],
            rnn_units=config['rnn_units'],
            seq_length=seq_length
        )
    
    model = compile_model(model, optimizer_name, learning_rate)
    
    # Training callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
    
    # Train
    print(f"\nTraining...")
    start_time = time.time()
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )
    
    training_time = time.time() - start_time
    
    # Evaluate on test set
    print(f"\n📊 Evaluating on test set...")
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    test_perplexity = calculate_perplexity(test_loss)
    
    # Get final training metrics
    final_train_loss = history.history['loss'][-1]
    final_train_acc = history.history['accuracy'][-1]
    final_val_loss = history.history['val_loss'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    
    # Store results
    result = {
        'Experiment_ID': f"{model_type}_{optimizer_name}" if not experiment_name else experiment_name,
        'Model': model_type.upper(),
        'Optimizer': optimizer_name.upper(),
        'Hyperparams': f"lr={learning_rate}, batch={batch_size}, epochs={epochs}",
        'Training_Time': round(training_time, 2),
        'Final_Train_Loss': round(final_train_loss, 4),
        'Final_Train_Accuracy': round(final_train_acc, 4),
        'Final_Val_Loss': round(final_val_loss, 4),
        'Final_Val_Accuracy': round(final_val_acc, 4),
        'Test_Loss': round(test_loss, 4),
        'Test_Accuracy': round(test_acc, 4),
        'Test_Perplexity': round(test_perplexity, 4),
        'Epochs_Trained': len(history.history['loss'])
    }
    
    all_experiment_results.append(result)
    
    # Print summary
    print(f"\nCOMPLETED")
    print(f"Training Time: {training_time:.2f}s ({training_time/60:.2f} min)")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Perplexity: {test_perplexity:.4f}")
    print("=" * 80)
    
    return model, history, result

print("Training utilities loaded!")
print("All experiment results will be stored in 'all_experiment_results' list")

Training utilities loaded!
All experiment results will be stored in 'all_experiment_results' list


### STEP 4.1: Training Utilities

Helper functions for training and evaluation.

### STEP 4.2: Experiment 1 - SimpleRNN + Adam

In [42]:
model_rnn_adam, history_rnn_adam, result_rnn_adam = train_and_evaluate(
    model_type='rnn',
    optimizer_name='adam',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: RNN + ADAM

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building RNN model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 27s 10ms/step - accuracy: 0.1890 - loss: 5.2961 - val_accuracy: 0.1920 - val_loss: 5.1417
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1914 - loss: 5.1918 - val_accuracy: 0.1920 - val_loss: 5.1361
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1935 - loss: 5.1825 - val_accuracy: 0.1920 - val_loss: 5.1423
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1889 - loss: 5.1946 - val_accuracy: 0.1920 - val_loss: 5.1376
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1914 - loss: 5.1876 - val_accuracy: 0.1920 - val_loss: 5.1378

📊 Evaluating on test set...

COMPLETED
Training Time: 82.72s (1.38 min)
Test Accuracy: 0.1868
Test Loss: 5.1865
T

### STEP 4.3: Experiment 2 - SimpleRNN + RMSprop

In [43]:
model_rnn_rmsprop, history_rnn_rmsprop, result_rnn_rmsprop = train_and_evaluate(
    model_type='rnn',
    optimizer_name='rmsprop',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: RNN + RMSPROP

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building RNN model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - accuracy: 0.1916 - loss: 5.2760 - val_accuracy: 0.1920 - val_loss: 5.4202
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1915 - loss: 5.2319 - val_accuracy: 0.1920 - val_loss: 5.3044
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1910 - loss: 5.2202 - val_accuracy: 0.1920 - val_loss: 5.1377
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1910 - loss: 5.1937 - val_accuracy: 0.1920 - val_loss: 5.1339
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1926 - loss: 5.1750 - val_accuracy: 0.1920 - val_loss: 5.1376
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.1910 - loss: 5.1458 - val_accuracy: 0.1920

### STEP 4.4: Experiment 3 - SimpleRNN + SGD

In [44]:
model_rnn_sgd, history_rnn_sgd, result_rnn_sgd = train_and_evaluate(
    model_type='rnn',
    optimizer_name='sgd',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: RNN + SGD

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building RNN model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - accuracy: 0.1751 - loss: 5.6583 - val_accuracy: 0.1920 - val_loss: 5.1475
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1895 - loss: 5.2003 - val_accuracy: 0.1920 - val_loss: 5.1307
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1903 - loss: 5.1736 - val_accuracy: 0.1920 - val_loss: 5.1272
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1902 - loss: 5.1688 - val_accuracy: 0.1920 - val_loss: 5.1260
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1919 - loss: 5.1486 - val_accuracy: 0.1920 - val_loss: 5.1246
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.1912 - loss: 5.1487 - val_accuracy: 0.1920 - v

### STEP 4.5: Experiment 4 - LSTM + Adam

In [45]:
model_lstm_adam, history_lstm_adam, result_lstm_adam = train_and_evaluate(
    model_type='lstm',
    optimizer_name='adam',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: LSTM + ADAM

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building LSTM model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - accuracy: 0.1880 - loss: 5.2650 - val_accuracy: 0.1928 - val_loss: 5.0073
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.1927 - loss: 5.0205 - val_accuracy: 0.1973 - val_loss: 4.9597
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.1985 - loss: 4.9459 - val_accuracy: 0.2036 - val_loss: 4.8980
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.2052 - loss: 4.8656 - val_accuracy: 0.2042 - val_loss: 4.8623
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.2072 - loss: 4.8214 - val_accuracy: 0.2050 - val_loss: 4.8355
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.2069 - loss: 4.7745 - val_accuracy: 0.

### STEP 4.6: Experiment 5 - LSTM + RMSprop

In [46]:
model_lstm_rmsprop, history_lstm_rmsprop, result_lstm_rmsprop = train_and_evaluate(
    model_type='lstm',
    optimizer_name='rmsprop',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: LSTM + RMSPROP

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building LSTM model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - accuracy: 0.1912 - loss: 5.2560 - val_accuracy: 0.1920 - val_loss: 5.1511
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.1907 - loss: 5.1276 - val_accuracy: 0.1932 - val_loss: 5.0219
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.1932 - loss: 5.0060 - val_accuracy: 0.1938 - val_loss: 4.9587
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.1957 - loss: 4.9334 - val_accuracy: 0.2003 - val_loss: 4.9033
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.2038 - loss: 4.8749 - val_accuracy: 0.2037 - val_loss: 4.8621
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.2064 - loss: 4.8116 - val_accuracy:

### STEP 4.7: Experiment 6 - LSTM + SGD

In [47]:
model_lstm_sgd, history_lstm_sgd, result_lstm_sgd = train_and_evaluate(
    model_type='lstm',
    optimizer_name='sgd',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: LSTM + SGD

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building LSTM model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.1876 - loss: 6.2726 - val_accuracy: 0.1920 - val_loss: 5.1669
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.1894 - loss: 5.2071 - val_accuracy: 0.1920 - val_loss: 5.1302
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.1926 - loss: 5.1556 - val_accuracy: 0.1920 - val_loss: 5.1232
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.1897 - loss: 5.1606 - val_accuracy: 0.1920 - val_loss: 5.1265
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.1905 - loss: 5.1535 - val_accuracy: 0.1920 - val_loss: 5.1245
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.1912 - loss: 5.1474 - val_accuracy: 0.1920 - 

### STEP 4.8: Experiment 7 - Transformer + Adam

In [48]:
model_transformer_adam, history_transformer_adam, result_transformer_adam = train_and_evaluate(
    model_type='transformer',
    optimizer_name='adam',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: TRANSFORMER + ADAM

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building TRANSFORMER model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 37s 11ms/step - accuracy: 0.1896 - loss: 5.2425 - val_accuracy: 0.1920 - val_loss: 5.1234
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1921 - loss: 5.1184 - val_accuracy: 0.1920 - val_loss: 5.1203
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1896 - loss: 5.1084 - val_accuracy: 0.1920 - val_loss: 5.0962
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1929 - loss: 5.0570 - val_accuracy: 0.1920 - val_loss: 5.0903
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1903 - loss: 5.0630 - val_accuracy: 0.1920 - val_loss: 5.0971
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1905 - loss: 5.0482 - val_acc

### STEP 4.9: Experiment 8 - Transformer + RMSprop

In [49]:
model_transformer_rmsprop, history_transformer_rmsprop, result_transformer_rmsprop = train_and_evaluate(
    model_type='transformer',
    optimizer_name='rmsprop',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: TRANSFORMER + RMSPROP

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building TRANSFORMER model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 37s 11ms/step - accuracy: 0.1895 - loss: 5.2152 - val_accuracy: 0.1917 - val_loss: 5.0448
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2019 - loss: 4.8986 - val_accuracy: 0.2053 - val_loss: 4.8443
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.2093 - loss: 4.7735 - val_accuracy: 0.2042 - val_loss: 4.8322
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2108 - loss: 4.7470 - val_accuracy: 0.2054 - val_loss: 4.8483
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.2150 - loss: 4.7234 - val_accuracy: 0.2042 - val_loss: 4.8145
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.2167 - loss: 4.7052 - val_

### STEP 4.10: Experiment 9 - Transformer + SGD

In [50]:
model_transformer_sgd, history_transformer_sgd, result_transformer_sgd = train_and_evaluate(
    model_type='transformer',
    optimizer_name='sgd',
    train_sequences=train_sequences,
    val_sequences=val_sequences,
    test_sequences=test_sequences,
    token_to_idx=token_to_idx
)

EXPERIMENT: TRANSFORMER + SGD

Preparing data...
Training samples: 120070, Input shape: (120070, 20)
Training samples: 15271, Input shape: (15271, 20)
Training samples: 14232, Input shape: (14232, 20)

Building TRANSFORMER model...

Training...
Epoch 1/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 30s 10ms/step - accuracy: 0.1865 - loss: 5.5081 - val_accuracy: 0.1920 - val_loss: 5.1388
Epoch 2/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1921 - loss: 5.1520 - val_accuracy: 0.1920 - val_loss: 5.1261
Epoch 3/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1919 - loss: 5.1318 - val_accuracy: 0.1920 - val_loss: 5.1191
Epoch 4/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1913 - loss: 5.1346 - val_accuracy: 0.1920 - val_loss: 5.1243
Epoch 5/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1912 - loss: 5.1276 - val_accuracy: 0.1920 - val_loss: 5.1190
Epoch 6/20
1877/1877 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.1926 - loss: 5.1306 - val_accu

### STEP 4.11: Core Experiments Summary

View all 9 experiment results.

In [51]:
# Display core experiments summary
import pandas as pd

print("🎯 CORE EXPERIMENTS SUMMARY")
print("=" * 80)
print(f"Total experiments completed: {len(all_experiment_results)}")
print("=" * 80)

if all_experiment_results:
    # Create DataFrame
    df_results = pd.DataFrame(all_experiment_results)
    
    # Display summary table
    print("\n📊 Results Table:")
    print(df_results[['Experiment_ID', 'Test_Accuracy', 'Test_Perplexity', 'Training_Time']].to_string(index=False))
    
    # Find best model
    best_idx = df_results['Test_Accuracy'].idxmax()
    best_model = df_results.iloc[best_idx]
    
    print(f"\n🏆 BEST MODEL: {best_model['Experiment_ID']}")
    print(f"   Accuracy: {best_model['Test_Accuracy']:.4f}")
    print(f"   Perplexity: {best_model['Test_Perplexity']:.4f}")
    print(f"   Training Time: {best_model['Training_Time']:.2f}s")
else:
    print("\n⚠️  No experiments completed yet. Run the experiment cells above.")

print("=" * 80)

🎯 CORE EXPERIMENTS SUMMARY
Total experiments completed: 14

📊 Results Table:
      Experiment_ID  Test_Accuracy  Test_Perplexity  Training_Time
           rnn_adam         0.1868         178.3094         128.27
        rnn_rmsprop         0.1868         176.0376         140.64
            rnn_sgd         0.1868         176.4497         101.93
           rnn_adam         0.1868         178.3588         110.48
        rnn_rmsprop         0.1868         175.9705         150.27
           rnn_adam         0.1868         178.8432          82.72
        rnn_rmsprop         0.1868         175.9400         149.17
            rnn_sgd         0.1868         176.4564         134.82
          lstm_adam         0.2065         120.5435         280.35
       lstm_rmsprop         0.2050         126.5593         190.11
           lstm_sgd         0.1868         176.8076         105.39
   transformer_adam         0.1960         141.7023         241.70
transformer_rmsprop         0.2057         125.4443 

## STEP 5: Hyperparameter Experimentation

Systematic hyperparameter tuning using One-Factor-At-A-Time (OFAT) methodology.

**OFAT Methodology:**
- Start with the best model from Step 4 as baseline
- Change one hyperparameter at a time while keeping others fixed
- Test specified values from syllabus for each parameter

**Parameters to Tune:**
1. **Architecture:** `num_layers` [1, 2, 3], `dropout_rate` [0.1, 0.2, 0.3, 0.5]
2. **Training:** `learning_rate` [0.0001, 0.001, 0.01, 0.1], `batch_size` [32, 64, 128, 256], `epochs` [10, 20, 30, 50], `seq_length` [10, 15, 20, 30]
3. **Transformer-specific (if best model is Transformer):** `num_heads` [2, 4, 8], `ff_dim` [256, 512, 1024], `num_blocks` [1, 2, 3, 4]

In [ ]:
import time
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Initialize hyperparameter results storage
if 'hyperparam_results' not in globals():
    hyperparam_results = []

print("="*80)
print("STEP 5: HYPERPARAMETER EXPERIMENTATION - OFAT METHODOLOGY")
print("="*80)

# Step 1: Identify best model from Step 4
if 'all_experiment_results' in globals() and all_experiment_results:
    df_core = pd.DataFrame(all_experiment_results)
    best_idx = df_core['Test_Accuracy'].idxmax()
    best_config = df_core.iloc[best_idx]
    
    print("\nBest Model from Core Experiments:")
    print(f"  Experiment ID: {best_config['Experiment_ID']}")
    print(f"  Model: {best_config['Model']}")
    print(f"  Optimizer: {best_config['Optimizer']}")
    print(f"  Test Accuracy: {best_config['Test_Accuracy']:.4f}")
    print(f"  Test Perplexity: {best_config['Test_Perplexity']:.4f}")
    print(f"  Training Time: {best_config['Training_Time']:.2f}s")
    
    # Extract baseline configuration
    best_model_type = best_config['Model'].lower()
    best_optimizer = best_config['Optimizer'].lower()
    
    # Get baseline config from MODEL_CONFIGS
    baseline_config = MODEL_CONFIGS[best_model_type].copy()
    baseline_config['optimizer'] = best_optimizer
    baseline_config['batch_size'] = 64  # Default from Step 4
    baseline_config['epochs'] = 10      # Default from Step 4
    
    print("\nBaseline Configuration:")
    for key, value in baseline_config.items():
        print(f"  {key}: {value}")
    
else:
    print("\nERROR: No core experiment results found.")
    print("Please run STEP 4 before running STEP 5.")
    best_model_type = None
    best_optimizer = None
    baseline_config = None

print("\n" + "="*80)

In [ ]:
# Helper function for OFAT experiments
def run_ofat_experiment(param_name, param_value, config, model_type, optimizer_name, 
                        X_train, y_train, X_val, y_val, X_test, y_test):
    """
    Run a single OFAT experiment with specified parameter value.
    
    Args:
        param_name: Name of the parameter being tuned
        param_value: Value to test for the parameter
        config: Configuration dictionary with all parameters
        model_type: Model architecture type
        optimizer_name: Optimizer to use
        X_train, y_train, X_val, y_val, X_test, y_test: Training data
        
    Returns:
        Dictionary with experiment results
    """
    print(f"\n  Testing {param_name}={param_value}...", end=" ")
    
    try:
        # Apply parameter value to config
        test_config = config.copy()
        test_config[param_name] = param_value
        
        # Special handling for different parameters
        batch_size = test_config.get('batch_size', 64)
        epochs = test_config.get('epochs', 10)
        learning_rate = test_config.get('learning_rate', 0.001)
        
        # Create model based on architecture
        with strategy.scope():
            vocab_size = len(token_to_idx)
            
            if model_type == 'transformer':
                model = create_transformer_model(
                    vocab_size=vocab_size,
                    embedding_dim=test_config.get('embedding_dim', 128),
                    num_heads=test_config.get('num_heads', 4),
                    ff_dim=test_config.get('ff_dim', 256),
                    num_blocks=test_config.get('num_blocks', 2),
                    seq_length=test_config.get('seq_length', 20),
                    dropout_rate=test_config.get('dropout_rate', 0.05)
                )
            else:
                # For RNN/LSTM with num_layers support
                if param_name == 'num_layers':
                    num_layers = param_value
                    dropout_rate = test_config.get('dropout_rate', 0.1)
                    
                    model = Sequential([Embedding(vocab_size, test_config['embedding_dim'], 
                                                 input_length=test_config['seq_length'], mask_zero=True)])
                    
                    # Add RNN/LSTM layers
                    for i in range(num_layers):
                        return_sequences = (i < num_layers - 1)  # Last layer doesn't return sequences
                        if model_type == 'rnn':
                            model.add(SimpleRNN(test_config['rnn_units'], return_sequences=return_sequences,
                                              dropout=dropout_rate, recurrent_dropout=dropout_rate))
                        else:  # lstm
                            model.add(LSTM(test_config['rnn_units'], return_sequences=return_sequences,
                                         dropout=dropout_rate, recurrent_dropout=0.0))
                    
                    model.add(Dense(test_config['rnn_units'] // 2, activation='relu'))
                    model.add(Dropout(dropout_rate))
                    model.add(Dense(vocab_size, activation='softmax'))
                    
                elif param_name == 'dropout_rate':
                    # Rebuild model with new dropout
                    dropout_rate = param_value
                    num_layers = test_config.get('num_layers', 2)
                    
                    model = Sequential([Embedding(vocab_size, test_config['embedding_dim'],
                                                 input_length=test_config['seq_length'], mask_zero=True)])
                    
                    for i in range(num_layers):
                        return_sequences = (i < num_layers - 1)
                        if model_type == 'rnn':
                            model.add(SimpleRNN(test_config['rnn_units'], return_sequences=return_sequences,
                                              dropout=dropout_rate, recurrent_dropout=dropout_rate))
                        else:
                            model.add(LSTM(test_config['rnn_units'], return_sequences=return_sequences,
                                         dropout=dropout_rate, recurrent_dropout=0.0))
                    
                    model.add(Dense(test_config['rnn_units'] // 2, activation='relu'))
                    model.add(Dropout(dropout_rate))
                    model.add(Dense(vocab_size, activation='softmax'))
                else:
                    # Use standard create_model function
                    model = create_model(
                        model_type=model_type,
                        vocab_size=vocab_size,
                        embedding_dim=test_config.get('embedding_dim', 128),
                        rnn_units=test_config.get('rnn_units', 256),
                        seq_length=test_config.get('seq_length', 20),
                        num_heads=test_config.get('num_heads', 4),
                        ff_dim=test_config.get('ff_dim', 256),
                        num_blocks=test_config.get('num_blocks', 2)
                    )
            
            # Compile model
            model = compile_model(model, optimizer_name, learning_rate)
        
        # Train model
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, 
                                      restore_best_weights=True, verbose=0)
        
        start_time = time.time()
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping],
            verbose=0
        )
        training_time = time.time() - start_time
        
        # Evaluate on test set
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
        test_perplexity = np.exp(test_loss)
        
        print(f"Acc={test_accuracy:.4f}, Perp={test_perplexity:.2f}, Time={training_time:.1f}s")
        
        # Return results
        return {
            'Parameter': param_name,
            'Value': param_value,
            'Model': model_type.upper(),
            'Optimizer': optimizer_name.upper(),
            'Test_Accuracy': round(test_accuracy, 4),
            'Test_Loss': round(test_loss, 4),
            'Test_Perplexity': round(test_perplexity, 4),
            'Training_Time': round(training_time, 2),
            'Epochs_Trained': len(history.history['loss'])
        }
        
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return None

print("OFAT experiment function defined.")

### OFAT Experiment 1: Number of Layers

Test different numbers of layers for the RNN/LSTM architecture.

In [ ]:
if best_model_type and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 1: Number of Layers")
    print("="*80)
    print(f"Testing values: [1, 2, 3]")
    print(f"Baseline: num_layers=2 (default)")
    
    # Prepare data with baseline seq_length
    seq_length = baseline_config['seq_length']
    vocab_size = len(token_to_idx)
    X_train, y_train = prepare_training_data(train_sequences, seq_length, vocab_size)
    X_val, y_val = prepare_training_data(val_sequences, seq_length, vocab_size)
    X_test, y_test = prepare_training_data(test_sequences, seq_length, vocab_size)
    
    # Add num_layers to baseline config
    baseline_config['num_layers'] = 2  # Default value
    
    # Test each value
    for num_layers in [1, 2, 3]:
        result = run_ofat_experiment(
            param_name='num_layers',
            param_value=num_layers,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'num_layers'])} experiments")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 2: Dropout Rate

Test different dropout rates for regularization.

In [ ]:
if best_model_type and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 2: Dropout Rate")
    print("="*80)
    print(f"Testing values: [0.1, 0.2, 0.3, 0.5]")
    print(f"Baseline: dropout_rate=0.1")
    
    # Add dropout_rate to baseline config
    baseline_config['dropout_rate'] = 0.1  # Default value
    
    # Test each value
    for dropout_rate in [0.1, 0.2, 0.3, 0.5]:
        result = run_ofat_experiment(
            param_name='dropout_rate',
            param_value=dropout_rate,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'dropout_rate'])} experiments")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 3: Learning Rate

Test different learning rates for optimization.

In [ ]:
if best_model_type and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 3: Learning Rate")
    print("="*80)
    print(f"Testing values: [0.0001, 0.001, 0.01, 0.1]")
    print(f"Baseline: learning_rate={baseline_config['learning_rate']}")
    
    # Test each value
    for lr in [0.0001, 0.001, 0.01, 0.1]:
        result = run_ofat_experiment(
            param_name='learning_rate',
            param_value=lr,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'learning_rate'])} experiments")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 4: Batch Size

Test different batch sizes for training.

In [ ]:
if best_model_type and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 4: Batch Size")
    print("="*80)
    print(f"Testing values: [32, 64, 128, 256]")
    print(f"Baseline: batch_size={baseline_config['batch_size']}")
    
    # Test each value
    for batch_size in [32, 64, 128, 256]:
        result = run_ofat_experiment(
            param_name='batch_size',
            param_value=batch_size,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'batch_size'])} experiments")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 5: Training Epochs

Test different numbers of training epochs.

In [ ]:
if best_model_type and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 5: Training Epochs")
    print("="*80)
    print(f"Testing values: [10, 20, 30, 50]")
    print(f"Baseline: epochs={baseline_config['epochs']}")
    
    # Test each value
    for epochs in [10, 20, 30, 50]:
        result = run_ofat_experiment(
            param_name='epochs',
            param_value=epochs,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'epochs'])} experiments")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 6: Sequence Length

Test different sequence lengths. **Note:** This requires re-generating training data for each value.

In [ ]:
if best_model_type and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 6: Sequence Length")
    print("="*80)
    print(f"Testing values: [10, 15, 20, 30]")
    print(f"Baseline: seq_length={baseline_config['seq_length']}")
    print("Note: Re-generating training data for each sequence length...")
    
    # Test each value
    for seq_len in [10, 15, 20, 30]:
        print(f"\n  Preparing data for seq_length={seq_len}...", end=" ")
        
        # Re-generate training data with new sequence length
        X_train_seq, y_train_seq = prepare_training_data(train_sequences, seq_len, vocab_size)
        X_val_seq, y_val_seq = prepare_training_data(val_sequences, seq_len, vocab_size)
        X_test_seq, y_test_seq = prepare_training_data(test_sequences, seq_len, vocab_size)
        
        print("Done.")
        
        result = run_ofat_experiment(
            param_name='seq_length',
            param_value=seq_len,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train_seq, y_train=y_train_seq,
            X_val=X_val_seq, y_val=y_val_seq,
            X_test=X_test_seq, y_test=y_test_seq
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'seq_length'])} experiments")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 7: Transformer-Specific - Number of Attention Heads

**Only runs if the best model is a Transformer.**

In [ ]:
if best_model_type == 'transformer' and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 7: Number of Attention Heads (Transformer)")
    print("="*80)
    print(f"Testing values: [2, 4, 8]")
    print(f"Baseline: num_heads={baseline_config.get('num_heads', 4)}")
    
    # Reset data to baseline seq_length
    seq_length = baseline_config['seq_length']
    X_train, y_train = prepare_training_data(train_sequences, seq_length, vocab_size)
    X_val, y_val = prepare_training_data(val_sequences, seq_length, vocab_size)
    X_test, y_test = prepare_training_data(test_sequences, seq_length, vocab_size)
    
    # Test each value
    for num_heads in [2, 4, 8]:
        result = run_ofat_experiment(
            param_name='num_heads',
            param_value=num_heads,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'num_heads'])} experiments")
    print("="*80)
elif best_model_type and best_model_type != 'transformer':
    print("="*80)
    print("OFAT EXPERIMENT 7: Number of Attention Heads (Transformer)")
    print("="*80)
    print(f"Skipping: Best model is {best_model_type.upper()}, not Transformer.")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 8: Transformer-Specific - Feed-Forward Dimension

**Only runs if the best model is a Transformer.**

In [ ]:
if best_model_type == 'transformer' and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 8: Feed-Forward Dimension (Transformer)")
    print("="*80)
    print(f"Testing values: [256, 512, 1024]")
    print(f"Baseline: ff_dim={baseline_config.get('ff_dim', 256)}")
    
    # Test each value
    for ff_dim in [256, 512, 1024]:
        result = run_ofat_experiment(
            param_name='ff_dim',
            param_value=ff_dim,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'ff_dim'])} experiments")
    print("="*80)
elif best_model_type and best_model_type != 'transformer':
    print("="*80)
    print("OFAT EXPERIMENT 8: Feed-Forward Dimension (Transformer)")
    print("="*80)
    print(f"Skipping: Best model is {best_model_type.upper()}, not Transformer.")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Experiment 9: Transformer-Specific - Number of Transformer Blocks

**Only runs if the best model is a Transformer.**

In [ ]:
if best_model_type == 'transformer' and baseline_config:
    print("="*80)
    print("OFAT EXPERIMENT 9: Number of Transformer Blocks")
    print("="*80)
    print(f"Testing values: [1, 2, 3, 4]")
    print(f"Baseline: num_blocks={baseline_config.get('num_blocks', 2)}")
    
    # Test each value
    for num_blocks in [1, 2, 3, 4]:
        result = run_ofat_experiment(
            param_name='num_blocks',
            param_value=num_blocks,
            config=baseline_config,
            model_type=best_model_type,
            optimizer_name=best_optimizer,
            X_train=X_train, y_train=y_train,
            X_val=X_val, y_val=y_val,
            X_test=X_test, y_test=y_test
        )
        
        if result:
            hyperparam_results.append(result)
    
    print("\n" + "="*80)
    print(f"Completed: {len([r for r in hyperparam_results if r['Parameter'] == 'num_blocks'])} experiments")
    print("="*80)
elif best_model_type and best_model_type != 'transformer':
    print("="*80)
    print("OFAT EXPERIMENT 9: Number of Transformer Blocks")
    print("="*80)
    print(f"Skipping: Best model is {best_model_type.upper()}, not Transformer.")
    print("="*80)
else:
    print("Skipping: Best model configuration not available.")

### OFAT Summary: Analysis and Export

Display all hyperparameter tuning results and export to CSV.

In [ ]:
if hyperparam_results:
    print("="*80)
    print("HYPERPARAMETER TUNING SUMMARY - OFAT METHODOLOGY")
    print("="*80)
    
    # Create DataFrame
    df_hyperparam = pd.DataFrame(hyperparam_results)
    
    print(f"\nTotal experiments completed: {len(df_hyperparam)}")
    print(f"Parameters tuned: {df_hyperparam['Parameter'].unique().tolist()}")
    
    # Display all results
    print("\n" + "="*80)
    print("All Hyperparameter Tuning Results:")
    print("="*80)
    print(df_hyperparam[['Parameter', 'Value', 'Test_Accuracy', 'Test_Perplexity', 
                         'Training_Time']].to_string(index=False))
    
    # Find best value for each parameter
    print("\n" + "="*80)
    print("Best Value for Each Parameter:")
    print("="*80)
    
    for param in df_hyperparam['Parameter'].unique():
        param_df = df_hyperparam[df_hyperparam['Parameter'] == param]
        best_idx = param_df['Test_Accuracy'].idxmax()
        best_row = param_df.loc[best_idx]
        
        print(f"\n{param}:")
        print(f"  Best Value: {best_row['Value']}")
        print(f"  Test Accuracy: {best_row['Test_Accuracy']:.4f}")
        print(f"  Test Perplexity: {best_row['Test_Perplexity']:.2f}")
        print(f"  Training Time: {best_row['Training_Time']:.2f}s")
    
    # Overall best configuration
    overall_best_idx = df_hyperparam['Test_Accuracy'].idxmax()
    overall_best = df_hyperparam.iloc[overall_best_idx]
    
    print("\n" + "="*80)
    print("Overall Best Single Configuration from OFAT:")
    print("="*80)
    print(f"Parameter: {overall_best['Parameter']}")
    print(f"Value: {overall_best['Value']}")
    print(f"Model: {overall_best['Model']}")
    print(f"Optimizer: {overall_best['Optimizer']}")
    print(f"Test Accuracy: {overall_best['Test_Accuracy']:.4f}")
    print(f"Test Perplexity: {overall_best['Test_Perplexity']:.2f}")
    print(f"Training Time: {overall_best['Training_Time']:.2f}s")
    
    # Export to CSV
    csv_path = 'hyperparameter_tuning_results.csv'
    df_hyperparam.to_csv(csv_path, index=False)
    print(f"\n" + "="*80)
    print(f"Results exported to: {csv_path}")
    print("="*80)
    
    print("\nSTEP 5 COMPLETE: Hyperparameter experimentation finished successfully.")
    
else:
    print("="*80)
    print("No hyperparameter tuning results available.")
    print("Please run the OFAT experiments above.")
    print("="*80)

## STEP 6: Text Generation

Generate Urdu poetry using trained models with specific seed words and temperature variations.

**Requirements:**
- **Seed Words (Urdu):**
  - محبت (Love)
  - دل (Heart)
  - شام (Evening)
  - یاد (Memory)
  - خوشی (Happiness)

- **Temperature Settings:** 0.7 (Conservative), 1.0 (Balanced), 1.3 (Creative)
- **Models:** Generate text from all 9 trained models
- **Output Length:** 30-50 tokens per generation

In [ ]:
# Comprehensive Text Generation with All Required Seed Words and Temperatures

# Define requirements
SEED_WORDS = [
    'محبت',  # Love
    'دل',    # Heart
    'شام',   # Evening  
    'یاد',   # Memory
    'خوشی'   # Happiness
]

TEMPERATURES = [0.7, 1.0, 1.3]  # Conservative, Balanced, Creative
MAX_LENGTH = 50  # tokens per generation

# Store all generation results
generation_results = []

# Get all trained models
trained_models = [
    ('SimpleRNN + Adam', model_rnn_adam if 'model_rnn_adam' in globals() else None),
    ('SimpleRNN + RMSprop', model_rnn_rmsprop if 'model_rnn_rmsprop' in globals() else None),
    ('SimpleRNN + SGD', model_rnn_sgd if 'model_rnn_sgd' in globals() else None),
    ('LSTM + Adam', model_lstm_adam if 'model_lstm_adam' in globals() else None),
    ('LSTM + RMSprop', model_lstm_rmsprop if 'model_lstm_rmsprop' in globals() else None),
    ('LSTM + SGD', model_lstm_sgd if 'model_lstm_sgd' in globals() else None),
    ('Transformer + Adam', model_transformer_adam if 'model_transformer_adam' in globals() else None),
    ('Transformer + RMSprop', model_transformer_rmsprop if 'model_transformer_rmsprop' in globals() else None),
    ('Transformer + SGD', model_transformer_sgd if 'model_transformer_sgd' in globals() else None),
]

print("=" * 80)
print("🎨 TEXT GENERATION - Urdu Poetry")
print("=" * 80)
print(f"\nGenerating with {len(SEED_WORDS)} seed words × {len(TEMPERATURES)} temperatures")
print(f"per model = {len(SEED_WORDS) * len(TEMPERATURES)} generations per model\n")

# Generate text for each model
for model_name, model in trained_models:
    if model is None:
        print(f"\n⚠️  {model_name}: Model not trained yet, skipping...")
        continue
    
    print(f"\n{'='*80}")
    print(f"📝 {model_name}")
    print(f"{'='*80}")
    
    for seed in SEED_WORDS:
        print(f"\n🌱 Seed: {seed}")
        print("-" * 60)
        
        for temp in TEMPERATURES:
            try:
                generated = generate_text(
                    model=model,
                    idx_to_token=idx_to_token,
                    token_to_idx=token_to_idx,
                    seed_text=seed,
                    max_length=MAX_LENGTH,
                    temperature=temp
                )
                
                # Store result
                generation_results.append({
                    'Model': model_name,
                    'Seed_Word': seed,
                    'Temperature': temp,
                    'Generated_Text': generated,
                    'Length': len(generated.split())
                })
                
                # Display
                print(f"  T={temp}: {generated[:80]}...")
                
            except Exception as e:
                print(f"  T={temp}: Error - {e}")

print(f"\n{'='*80}")
print(f"✅ Text Generation Complete!")
print(f"Total generations: {len(generation_results)}")
print(f"{'='*80}")

## STEP 7: Results Export & Visualization

Export all experiment results to CSV and create visualizations for analysis.

In [ ]:
# Export all experiment results to CSV
import pandas as pd
import matplotlib.pyplot as plt
import os

print("=" * 80)
print("💾 EXPORTING RESULTS")
print("=" * 80)

# 1. Export Core Experiment Results
if all_experiment_results:
    df_experiments = pd.DataFrame(all_experiment_results)
    
    # Save to CSV
    csv_path = 'data/experiment_results.csv'
    df_experiments.to_csv(csv_path, index=False)
    print(f"\n✅ Core experiments exported to: {csv_path}")
    print(f"   Columns: {list(df_experiments.columns)}")
    print(f"   Rows: {len(df_experiments)}")
    
    # Display summary
    print(f"\n📊 RESULTS SUMMARY:")
    print(df_experiments[['Experiment_ID', 'Test_Accuracy', 'Test_Perplexity', 'Training_Time']].to_string(index=False))
else:
    print("\n⚠️  No core experiment results to export")

# 2. Export Text Generation Results
if 'generation_results' in globals() and generation_results:
    df_generation = pd.DataFrame(generation_results)
    gen_csv_path = 'data/text_generation_results.csv'
    df_generation.to_csv(gen_csv_path, index=False)
    print(f"\n✅ Text generations exported to: {gen_csv_path}")
    print(f"   Total generations: {len(df_generation)}")
else:
    print("\n⚠️  No text generation results to export")

# 3. Export Hyperparameter Tuning Results (if available)
if 'hyperparam_results' in globals() and hyperparam_results:
    df_hyperparam = pd.DataFrame(hyperparam_results)
    hyper_csv_path = 'data/hyperparameter_tuning_results.csv'
    df_hyperparam.to_csv(hyper_csv_path, index=False)
    print(f"\n✅ Hyperparameter tuning exported to: {hyper_csv_path}")
else:
    print("\n⚠️  No hyperparameter tuning results to export")

print("\n" + "=" * 80)
print("📈 CREATING VISUALIZATIONS")
print("=" * 80)

# Create visualizations if we have results
if all_experiment_results:
    # Set up plotting style
    plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available else 'default')
    
    # Figure 1: Test Accuracy Comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    df_plot = pd.DataFrame(all_experiment_results)
    models = df_plot['Experiment_ID']
    accuracies = df_plot['Test_Accuracy']
    perplexities = df_plot['Test_Perplexity']
    
    # Plot 1: Accuracy
    bars1 = ax1.barh(models, accuracies, color='skyblue')
    ax1.set_xlabel('Test Accuracy', fontsize=12)
    ax1.set_title('Model Comparison - Test Accuracy', fontsize=14, fontweight='bold')
    ax1.set_xlim([0, max(accuracies) * 1.1])
    
    # Add value labels
    for bar in bars1:
        width = bar.get_width()
        ax1.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.4f}', ha='left', va='center', fontsize=9)
    
    # Plot 2: Perplexity
    bars2 = ax2.barh(models, perplexities, color='lightcoral')
    ax2.set_xlabel('Test Perplexity', fontsize=12)
    ax2.set_title('Model Comparison - Test Perplexity (Lower is Better)', fontsize=14, fontweight='bold')
    
    # Add value labels
    for bar in bars2:
        width = bar.get_width()
        ax2.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.2f}', ha='left', va='center', fontsize=9)
    
    plt.tight_layout()
    viz_path = 'data/model_comparison.png'
    plt.savefig(viz_path, dpi=300, bbox_inches='tight')
    print(f"\n✅ Visualization saved to: {viz_path}")
    plt.show()
    
    # Figure 2: Training Time Comparison
    fig2, ax3 = plt.subplots(figsize=(10, 6))
    train_times = df_plot['Training_Time']
    bars3 = ax3.barh(models, train_times, color='lightgreen')
    ax3.set_xlabel('Training Time (seconds)', fontsize=12)
    ax3.set_title('Model Training Time Comparison', fontsize=14, fontweight='bold')
    
    for bar in bars3:
        width = bar.get_width()
        ax3.text(width, bar.get_y() + bar.get_height()/2,
                f'{width:.1f}s', ha='left', va='center', fontsize=9)
    
    plt.tight_layout()
    time_viz_path = 'data/training_time_comparison.png'
    plt.savefig(time_viz_path, dpi=300, bbox_inches='tight')
    print(f"✅ Training time visualization saved to: {time_viz_path}")
    plt.show()
    
else:
    print("\n⚠️  No data available for visualization")

print("\n" + "=" * 80)
print("🎯 PROJECT COMPLETE!")
print("=" * 80)
print("\n📁 All results saved to data/ directory:")
print("  • experiment_results.csv - Core experiment metrics")
print("  • text_generation_results.csv - Generated poetry samples")
print("  • model_comparison.png - Accuracy & Perplexity charts")
print("  • training_time_comparison.png - Training duration analysis")
print("\n✅ Semester project notebook successfully completed!")
print("=" * 80)